In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('./')
from capture_board import *

In [ ]:
image_paths = sorted(glob.glob('./images/*'))
image_paths

In [ ]:
path = image_paths[0]
image = cv2.imread(path,0)
image.shape

In [ ]:
print(np.min(image), np.max(image))
plt.hist(image.ravel(), bins=51,range=[0,255])
plt.show()

In [ ]:
plt.imshow(image,cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
image_resized = cv2.resize(image, None, fx=0.3, fy=0.3)
plt.imshow(image_resized,cmap='gray', vmin=0, vmax=255)
plt.show()

# Canny edge

In [ ]:
# image_canny = cv2.Canny(image_resized,50,150,apertureSize = 3)

In [ ]:
# plt.figure(figsize=(30,20))
# plt.imshow(image_canny,cmap='gray', vmin=0, vmax=255)
# plt.show()

# Laplacian filter

In [ ]:
image_lap = cv2.Laplacian(image_resized, cv2.CV_32F, ksize=29)
print(np.min(image_lap), np.max(image_lap))

plt.hist(image_lap.ravel(), bins=51)
plt.show()

# plt.imshow(image_lap, cmap='gray')
# plt.show()

In [ ]:
lap_med, lap_max = np.median(image_lap.ravel()), np.max(image_lap.ravel())
image_lap_norm = (image_lap - lap_med)/(lap_max-lap_med)
image_lap_norm[image_lap_norm<0] = 0
image_lap_pow = pow(image_lap_norm, 1/2)
image_lap_pow = (image_lap_pow*255).astype(np.uint8)

plt.hist(image_lap_pow.ravel(), bins=51)
plt.show()

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(image_lap_pow>50, cmap='gray')
plt.show()

## Hough line

In [ ]:
# lines = cv2.HoughLines(image_lap_pow,1, np.pi/180,150).squeeze()
# print(len(lines))

# draw = cv2.cvtColor(image_lap_pow, cv2.COLOR_GRAY2RGB)
# for rho,theta in lines:
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a*rho
#     y0 = b*rho
#     x1 = int(x0 + 1000*(-b))
#     y1 = int(y0 + 1000*(a))
#     x2 = int(x0 - 1000*(-b))
#     y2 = int(y0 - 1000*(a))

#     cv2.line(draw,(x1,y1),(x2,y2),(0,0,255),2)

In [ ]:
# plt.figure(figsize=(30,20))
# plt.imshow(draw)
# plt.show()

# findContour

In [ ]:
from skimage.color import label2rgb

In [ ]:
retval, labels, stats, centroids = cv2.connectedComponentsWithStats((image_lap_pow>50).astype(np.uint8))

In [ ]:
border_labels = list(set(np.unique(labels[0,:])) | set(np.unique(labels[:,0])) | set(np.unique(labels[-1,:])) | set(np.unique(labels[:,-1])))

full_labels = [ label for label in range(len(stats)) if label not in border_labels ]
full_label_image = labels.copy()
for border in border_labels:
    full_label_image[full_label_image==border] = 0

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(label2rgb(full_label_image, bg_label=0))
plt.show()

In [ ]:
candidate_idx = np.argmax(stats[full_labels][:,-1])
candidate = stats[full_labels][candidate_idx]

In [ ]:
len(stats), candidate

In [ ]:
candidate_original_idx = np.array(list(range(len(stats))))[full_labels][candidate_idx]
frame = (label2rgb(full_label_image==candidate_original_idx, bg_label=0)*255).astype(np.uint8)
# cv2.rectangle(draw, (candidate[0], candidate[1]), (candidate[0]+candidate[2], candidate[1]+candidate[3]), (255,255,255), 3)

draw = (frame*0.5 + cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)*0.5).astype(np.uint8)
plt.figure(figsize=(30,20))
plt.imshow(draw)
plt.show()

In [ ]:
frame_gr = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

In [ ]:
corners = get_corners(frame_gr)

In [ ]:
corners

In [ ]:
draw = frame.copy()
for i, corner in enumerate(corners):
    print(corner)
    cv2.circle(draw, tuple(corner[::-1]), 8, (0,255,0), -1)
plt.figure(figsize=(30,20))
plt.imshow(draw)
plt.show()

In [ ]:
width = 28
height = 28
board_width = width*9
board_height = height*9

In [ ]:
src = np.float32([ corner for corner in corners ])

# 変換後の4点
dst = np.float32([[0, 0],[0, board_width],[board_height, 0],[board_height,board_width]])
 
print(src.shape, dst.shape)
# 変換行列
M = cv2.getPerspectiveTransform(src[:,::-1], dst[:,::-1])
 
# 射影変換・透視変換する
output = cv2.warpPerspective(image_resized,M,(board_width, board_height))
print(image_resized.shape, output.shape)

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(output, cmap='gray')
# plt.grid()
plt.show()

In [ ]:
# def get_cell(output,i):
#     j, k = i//9, i%9
#     cell = output[28*j:28*(j+1),28*k:28*(k+1)]
#     cell = 255 - (cell - np.min(cell))/(np.max(cell) - np.min(cell)) * 255
#     ret, cell = cv2.threshold(cell, 150, 255, cv2.THRESH_BINARY)
# #     cell = cv2.adaptiveThreshold(cell.astype(np.uint8),255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
# #             cv2.THRESH_BINARY,11,2)
# #     cell = cv2.morphologyEx(cell, cv2.MORPH_ERODE, (3,3), 1)
#     cell = cv2.morphologyEx(cell, cv2.MORPH_OPEN, (5,5), 1)
# #     cell = cv2.morphologyEx(cell, cv2.MORPH_CLOSE, (5,5), 1)
#     cell = cv2.morphologyEx(cell, cv2.MORPH_DILATE, (3,3), 1)
#     return cell

In [ ]:
plt.imshow(output)

In [ ]:
%%time
cells = np.array([ get_cell(output,i,width,height,50) for i in range(81) ])
cells.shape

In [ ]:
cells = capture_image(image_paths[0],width=28,height=28)
print(cells.shape)

In [ ]:
plt.figure(figsize=(6,6))
for i,cell in enumerate(cells,1):
    plt.subplot(9,9,i)
    plt.imshow(cell, cmap='gray', vmin=0)
    plt.axis('off')
plt.tight_layout()
plt.show()

# Annotation label

In [ ]:
labels = [
    0,1,2,0,0,0,9,8,0,
    7,0,0,2,0,8,0,0,4,
    8,0,0,0,9,0,0,0,2,
    0,2,8,0,0,0,6,3,0,
    0,0,0,6,0,5,0,1,0,
    0,4,6,0,0,9,0,7,0,
    3,0,0,0,1,0,0,2,0,
    0,6,5,9,0,0,8,0,0,
    0,0,0,0,6,2,0,0,0
]
labels = np.array(labels)

In [ ]:
for i in range(10):
    plt.figure(figsize=(10,3))
    for j,cell in enumerate(cells[labels==i][:10],1):
        plt.subplot(1,10,j)
        plt.imshow(cell, cmap='gray', vmin=0)
        plt.axis('off')
    plt.tight_layout()
plt.show()